# Location Analysis German

### Setup

**Important imports**

In [2]:
import pandas as pd
import spacy
import requests
import time
import re
import unicodedata

In [37]:
nlp = spacy.load("de_core_news_sm")

In [4]:
df_german = pd.read_csv('../../scraping/data/extractor_all_articles_20minuten.csv')
df_french = pd.read_csv('../../scraping/data/extractor_all_articles_20minutes.csv')

In [331]:
german_iso_3166 = pd.read_csv('inputdata/german-iso-3166.csv', names=['ISO2', 'Name'])
english_iso_3166 = pd.read_csv('inputdata/english-iso-3166.csv', usecols=['alpha-2', 'name', 'alpha-3'])
english_iso_3166.columns = ['Name', 'ISO2', 'ISO3']

**Loading dataframes and cleaning them**

In [199]:
def normalize_text(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii').lower()

In [ ]:
def apply_normalize_text_if_str(x):
    return normalize_text(x) if isinstance(x, str) else x

In [ ]:
def lowercase_columns(df, columns):
    for column in columns:
        df[column] = df[column].str.lower()

In [203]:
columns_to_normalize = ['Content', 'Title', 'Header']
df_german[columns_to_normalize] = df_german[columns_to_normalize].applymap(apply_normalize_text_if_str)

In [ ]:
df_countries_german = pd.read_csv('inputdata/country_capital_german.csv')
columns_to_lower = ['Staat', 'Hauptstadt', 'Kontinent']
lowercase_columns(df_countries_german, columns_to_lower)

In [193]:
df_cities_german = pd.read_csv('inputdata/listestaedte_schweiz.csv')
df_cities_german['Name'] = df_cities_german['Name'].apply(lambda x: normalize_text(x) if isinstance(x, str) else x)

In [62]:
df_cantons_german = pd.read_csv('inputdata/kantone.csv')
df_cantons_german['Kanton'] = df_cantons_german['Kanton'].apply(lambda x: normalize_text(x) if isinstance(x, str) else x)

### Add Coordinates Functionality

In [135]:
geocode_api_key = '1983e85e9a97673a09ed6d19417dda0f'

In [107]:
def geocode_location_positionstack(location_name, api_key):
    base_url = "http://api.positionstack.com/v1/forward"
    params = {
        'access_key': api_key,
        'query': location_name,
        'limit': 1,
    }
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['data']:
            latitude = data['data'][0]['latitude']
            longitude = data['data'][0]['longitude']
            return latitude, longitude
        else:
            return None, None
    else:
        print(f"Error: {response.status_code}")
        return None, None

In [ ]:
def add_coordinates(row):
    lat, lng = geocode_location_positionstack(row['Name'], geocode_api_key)
    return pd.Series({'Coordinates': (lat, lng), 'Latitude': lat, 'Longitude': lng})

In [137]:
def add_coordinates(row, column_name, api_key):
    location_name = row[column_name]
    lat, lng = geocode_location_positionstack(location_name, api_key)
    return pd.Series({'Coordinates': (lat, lng), 'Latitude': lat, 'Longitude': lng})

**Add coordinates to cities in Switzerland**

In [ ]:
#df_cities_german[['Coordinates', 'Latitude', 'Longitude']] = df_cities_german.apply(add_coordinates, axis=1)
df_cities_german[['Coordinates', 'Latitude', 'Longitude']] = df_cities_german.apply(lambda row: add_coordinates(row, 'Name', geocode_api_key), axis=1)

In [133]:
indices_with_none = df_cities_german[df_cities_german['Coordinates'] == (None, None)].index

In [130]:
for idx in indices_with_none:
    city_name = df_cities_german.loc[idx, 'Name']
    print(f"Attempting to geocode: {city_name}") 
    try:

        time.sleep(1)
        
        new_coords = geocode_location_positionstack(city_name, geocode_api_key)
        print(f"Coordinates received: {new_coords}")  
        
        if new_coords is None or len(new_coords) != 2:
            raise ValueError(f"Invalid coordinates received: {new_coords}")
        
        df_cities_german.loc[idx, 'Latitude'] = new_coords[0]
        df_cities_german.loc[idx, 'Longitude'] = new_coords[1]

        df_cities_german.at[idx, 'Coordinates'] = (new_coords[0], new_coords[1])

    except Exception as e:
        print(f"Error geocoding {city_name}: {e}")
        traceback.print_exc() 
        df_cities_german.loc[idx, ['Latitude', 'Longitude']] = [None, None]
        df_cities_german.at[idx, 'Coordinates'] = (None, None)

Attempting to geocode: val-de-travers ne
Coordinates received: (46.917395, 6.588048)
Attempting to geocode: sarnen ⁠
Coordinates received: (46.895035, 8.248186)
Attempting to geocode: flawil
Coordinates received: (47.41568, 9.189439)
Attempting to geocode: neuhausen am rheinfall
Coordinates received: (47.682077, 8.612612)
Attempting to geocode: aigle ⁠
Coordinates received: (46.317669, 6.970125)
Attempting to geocode: lutry ⁠
Coordinates received: (46.503453, 6.686783)
Attempting to geocode: birsfelden
Coordinates received: (47.550784, 7.636283)
Attempting to geocode: sursee ⁠
Coordinates received: (47.175697, 8.10871)
Attempting to geocode: aesch
Coordinates received: (47.47104, 7.5973)
Attempting to geocode: naters
Coordinates received: (46.326545, 7.987857)
Attempting to geocode: gossau
Coordinates received: (47.411712, 9.251598)
Attempting to geocode: bernex
Coordinates received: (46.174748, 6.077743)
Attempting to geocode: münchenbuchsee
Coordinates received: (47.022114, 7.451819)

**Add coordinates to capital cities worldwide**

In [139]:
df_countries_german[['Coordinates', 'Latitude', 'Longitude']] = df_countries_german.apply(lambda row: add_coordinates(row, 'Hauptstadt', geocode_api_key), axis=1)

### Search Functions

#### Country Search

In [90]:
def find_countries_in_text(content, header, country_data, swiss_cities, swiss_cantons):
    combined_text = str(content) + " " + str(header)
    doc = nlp(combined_text)
    mentioned_countries = set()

    for ent in doc.ents:
        if ent.label_ in ['LOC', 'GPE']:

            if ent.text.lower() in swiss_cities['Name'].str.lower().values or ent.text.lower() in swiss_cantons['Kanton'].str.lower().values:
                mentioned_countries.add("Schweiz")
                continue  

            country_from_capital = country_data[country_data['Hauptstadt'].str.lower() == ent.text.lower()]['Staat']
            if not country_from_capital.empty:
                mentioned_countries.add(country_from_capital.iloc[0])
                continue 

            country = country_data[country_data['Staat'].str.lower() == ent.text.lower()]['Staat']
            if not country.empty:
                mentioned_countries.add(country.iloc[0])

    return mentioned_countries

In [93]:
df_german['Mentioned_Countries'] = df_german.apply(lambda row: find_countries_in_text(row['Content'], row['Header'], df_countries_german, df_cities_german, df_cantons_german), axis=1)

#### Count Country Mentions

In [168]:
country_counts = df_german.explode('Mentioned_Countries')['Mentioned_Countries'].value_counts()
df_country_counts = pd.DataFrame({'Country': country_counts.index, 'Count': country_counts.values})
df_country_counts.sort_values(by='Count', ascending=False, inplace=True)

In [260]:
df_countries_german

,Staat,Hauptstadt,Kontinent,Coordinates,Latitude,Longitude
0,Afghanistan,Kabul,Asien,"(34.52813, 69.17233)",34.528130,69.172330
1,Ägypten,Kairo,Afrika,"(30.079694, 31.323437)",30.079694,31.323437
2,Albanien,Tirana,Europa,"(41.282045, 19.799067)",41.282045,19.799067
3,Algerien,Algier,Afrika,"(36.74395, 3.083627)",36.743950,3.083627
4,Andorra,Andorra la Vella,Europa,"(42.504294, 1.518714)",42.504294,1.518714
...,...,...,...,...,...,...
205,Südossetien,Zchinwali,NaN,"(42.22764, 43.96861)",42.227640,43.968610
206,Taiwan,Taipeh,NaN,"(25.04776, 121.53185)",25.047760,121.531850
207,Transnistrien,Tiraspol,NaN,"(46.848936, 29.630367)",46.848936,29.630367
208,Westsahara,El Aaiún,NaN,"(27.16224, -13.20315)",27.162240,-13.203150


In [333]:
df_country_count_data = df_country_counts.merge(df_countries_german, left_on='Country', right_on='Staat', how='left')
df_country_count_data = df_country_count_data[['Country', 'Count', 'Hauptstadt', 'Coordinates', 'Latitude', 'Longitude']]

In [334]:
df_country_count_data

,Country,Count,Hauptstadt,Coordinates,Latitude,Longitude
0,Schweiz,315,Bern,"(46.946416, 7.396394)",46.946416,7.396394
1,Deutschland,69,Berlin,"(52.524932, 13.407032)",52.524932,13.407032
2,Israel,49,Jerusalem,"(31.759595, 35.215315)",31.759595,35.215315
3,Italien,34,Rom,"(41.878243, 12.528091)",41.878243,12.528091
4,Frankreich,32,Paris,"(48.858705, 2.342865)",48.858705,2.342865
...,...,...,...,...,...,...
98,Sudan,1,Khartum,"(15.55177, 32.53241)",15.551770,32.532410
99,Mosambik,1,Maputo,"(-25.96553, 32.58322)",-25.965530,32.583220
100,Lettland,1,Riga,"(56.971149, 24.142749)",56.971149,24.142749
101,Mazedonien,1,Skopje,"(42.00122, 21.42878)",42.001220,21.428780


**Add translation so that we have German and English Names (based on ISO Codes)**

In [335]:
df_final_country_count_data = df_country_count_data.merge(german_iso_3166, left_on='Country', right_on='Name', how='left')
df_final_country_count_data = df_final_country_count_data[['Country', 'ISO2', 'Count', 'Hauptstadt', 'Coordinates', 'Longitude', 'Latitude']]


In [343]:
df_final_country_count_data = df_final_country_count_data.merge(english_iso_3166, on='ISO2', how='left')

# Rename the 'Name' column from english_iso_3166 to 'EnglishName'
df_final_country_count_data.rename(columns={'Name': 'EnglishName'}, inplace=True)
df_final_country_count_data.rename(columns={'Country': 'GermanName'}, inplace=True)
df_final_country_count_data.rename(columns={'Hauptstadt': 'Capital'}, inplace=True)

In [346]:
desired_columns = [
    'GermanName', 'Count', 'Coordinates', 'Longitude',
    'Latitude', 'ISO2', 'ISO3', 'EnglishName', 'Capital'
]

df_final_country_count_data = df_final_country_count_data[desired_columns]

#### City Search

In [218]:
def find_swiss_cities_in_text(content, header, df_cities_german):
    combined_text = normalize_text(str(content) + " " + str(header)).strip()
    mentioned_swiss_cities = set()

    for city in df_cities_german['Name']:
        normalized_city = normalize_text(city).strip()
        # Use a regular expression to find whole word matches only
        if re.search(r'\b' + re.escape(normalized_city) + r'\b', combined_text):
            mentioned_swiss_cities.add(city)

    return mentioned_swiss_cities

In [242]:
df_german['Mentioned_Swiss_Cities'] = df_german.apply(lambda row: find_swiss_cities_in_text(row['Content'], row['Header'], df_cities_german), axis=1)


In [244]:
switzerland_added = False  # Flag to track if 'Switzerland' is ever added

for idx, row in df_german.iterrows():
    if row['Mentioned_Swiss_Cities'] and "Schweiz" not in row['Mentioned_Countries']:
        df_german.at[idx, 'Mentioned_Countries'] = row['Mentioned_Countries'].union({"Schweiz"})
        print(f"Row {idx} updated: {row}")
        switzerland_added = True

# Check if 'Switzerland' was added in any of the rows
if not switzerland_added:
    print("No updates made. 'Schweiz' was already included in all relevant rows.")

Row 7 updated: Title                               museum begeht tabubruch  und hat erfolg
Header                                                                baden
Content                   an der diesjahrigen 20th century art evening a...
Mentioned_Countries                                               {Schweiz}
Mentioned_Swiss_Cities                                             {baden }
Name: 7, dtype: object
Row 14 updated: Title                     constantin plagen schuldgefuhle, er zeigt sich...
Header                                                         spitalbesuch
Content                   ende oktober machten die news die runde, dass ...
Mentioned_Countries                                               {Schweiz}
Mentioned_Swiss_Cities                                            {sitten }
Name: 14, dtype: object
Row 17 updated: Title                     fluchtiger morder erhebt schwere vorwurfe in a...
Header                                                          deutsc

In [264]:
city_counts = df_german.explode('Mentioned_Swiss_Cities')['Mentioned_Swiss_Cities'].value_counts()
df_city_counts = pd.DataFrame({'City': city_counts.index, 'Count': city_counts.values})
df_city_counts.sort_values(by='Count', ascending=False, inplace=True)

In [265]:
df_city_count_data = df_city_counts.merge(df_cities_german, left_on='City', right_on='Name', how='left')
df_city_count_data = df_city_count_data[['City', 'Count', 'Coordinates', 'Longitude', 'Latitude']]

### Exports

In [ ]:
csv_file_path = 'inputdata/df_cities_german_with_coordinates.csv'
df_cities_german.to_csv(csv_file_path, index=False)

In [141]:
csv_file_path = 'inputdata/df_countries_german_with_capital_and_coordinates.csv'
df_countries_german.to_csv(csv_file_path, index=False)

In [273]:
csv_file_path = 'data/df_german_with_mentions.csv'
df_german.to_csv(csv_file_path, index=False)

In [256]:
csv_file_path = 'data/df_city_count_data.csv'
df_city_count_data.to_csv(csv_file_path, index=False)

In [271]:
csv_file_path = 'data/df_country_count_data.csv'
df_country_count_data.to_csv(csv_file_path, index=False)

In [350]:
csv_file_path = 'data/df_translated_country_count_data.csv'
df_final_country_count_data.to_csv(csv_file_path, index=False)

### Debugs

In [ ]:
def find_swiss_cities_in_text_debug(content, header, df_cities_german):
    combined_text = (str(content) + " " + str(header)).lower()
    mentioned_swiss_cities = set()

    print("Combined Text:", combined_text)  # Debug print

    for city in df_cities_german['Name']:
        city_lower = city.lower()
        print(f"Checking for city: {city_lower}")  # Debug print
        if city_lower in combined_text:
            mentioned_swiss_cities.add(city)

    return mentioned_swiss_cities

# Test the function with the first row's content and header
mentioned_cities_debug = find_swiss_cities_in_text_debug(first_row_content, first_row_header, df_cities_german)
print("Mentioned Swiss cities in the first row (Debug):", mentioned_cities_debug)

In [ ]:
import unicodedata

def normalize_text(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii').lower()

def find_swiss_cities_in_text_debug(content, header, df_cities_german):
    combined_text = normalize_text(str(content) + " " + str(header))
    mentioned_swiss_cities = set()

    print("Normalized Combined Text:", combined_text)  # Debug print

    for city in df_cities_german['Name']:
        normalized_city = normalize_text(city)
        print(f"Checking for city: {normalized_city}")  # Debug print
        if normalized_city in combined_text:
            mentioned_swiss_cities.add(city)

    return mentioned_swiss_cities

# Test the function with the first row's content and header
mentioned_cities_debug = find_swiss_cities_in_text_debug(first_row_content, first_row_header, df_cities_german)
print("Mentioned Swiss cities in the first row (Debug):", mentioned_cities_debug)

In [ ]:
def find_swiss_cities_in_text(content, header, df_cities_german):
    combined_text = normalize_text(str(content) + " " + str(header))
    print("Normalized Combined Text:", combined_text)  # Debug print

    mentioned_swiss_cities = set()

    normalized_swiss_city_names = set(df_cities_german['Name'].apply(normalize_text))
    print("Normalized Swiss City Names:", normalized_swiss_city_names)  # Debug print

    for city in normalized_swiss_city_names:
        print(f"Checking for city: {city}")  # Debug print
        if city in combined_text:
            mentioned_swiss_cities.add(city)

    return mentioned_swiss_cities

# Testing the function with a single row (e.g., first row) from df_german
test_row = df_german.iloc[0]
test_mentioned_cities = find_swiss_cities_in_text(test_row['Content'], test_row['Header'], df_cities_german)
print("Mentioned Swiss cities in test row:", test_mentioned_cities)


In [ ]:
test_row = df_german.iloc[0]
